# Memetic Algorithm for Text Evolution (MATE)

This is where I code and try out MATE. MATE has been explained significantly in [README.md](README.md). Go through that to know how I plan to use a memetic algorithm to jailbreak our tier-c model :)

In [38]:
# Cell 1: Imports and Setup
import os
import glob
import shutil
import torch
import numpy as np
from pathlib import Path
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from dotenv import load_dotenv
from google import genai
import time
from sentence_transformers import SentenceTransformer
import shap
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import json

# Setup directories
POPULATION_DIR = Path("population")
EVOLUTION_DIR = Path("evolution")
INPUT_FILE = Path("input.txt")
MODEL_PATH = Path("../task-2/transformer/tier_c_final_model")

print("Directories configured")

Directories configured


In [39]:
# Cell 2: Load Classifier Model
print("Loading classifier model...")

# Load PEFT configuration
peft_model_id = str(MODEL_PATH)
config = PeftConfig.from_pretrained(peft_model_id)

# Load base model
base_model = AutoModelForSequenceClassification.from_pretrained(
    config.base_model_name_or_path,  # "distilbert-base-uncased"
    num_labels=3
)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model = PeftModel.from_pretrained(base_model, peft_model_id)
model.eval()

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"Model loaded on {device}")
print(f"  Base: {config.base_model_name_or_path}")

Loading classifier model...


Loading weights: 100%|██████████| 100/100 [00:00<00:00, 1196.62it/s, Materializing param=distilbert.transformer.layer.5.sa_layer_norm.weight]   
DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_transform.weight  | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
pre_classifier.bias     | MISSING    | 
pre_classifier.weight   | MISSING    | 
classifier.weight       | MISSING    | 
classifier.bias         | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Model loaded on cpu
  Base: distilbert-base-uncased


In [40]:
# Cell 3: Initialize Gemini API
print("Initializing Gemini API...")

# Load API key
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

if not api_key:
    raise ValueError("GEMINI_API_KEY not found in .env file!")

# Initialize client
client = genai.Client(api_key=api_key)
MODEL_NAME = "gemini-pro-latest" 

print(f"Gemini API initialized")
print(f"Model: {MODEL_NAME}")

Initializing Gemini API...
Gemini API initialized
Model: gemini-pro-latest


In [41]:
# Cell 3.5: [ DEBUG ] Test Gemini API - FIXED
print("Testing Gemini API with CORRECTED max_output_tokens...")

test_prompt = "Write a short paragraph about technology."

try:
    response = client.models.generate_content(
        model=MODEL_NAME,
        contents=test_prompt,
        config={
            "temperature": 0.7,
            "top_p": 0.95,
            "max_output_tokens": 8192,  # FIXED: Was 512, causing MAX_TOKENS error
        }
    )
    
    print(f"Response received successfully!")
    
    if response.candidates:
        candidate = response.candidates[0]
        print(f"\nFinish reason: {candidate.finish_reason}")
        
        if candidate.content and candidate.content.parts:
            raw_text = candidate.content.parts[0].text
            print(f"\nGenerated text ({len(raw_text)} chars):")
            print(raw_text)
            print("\n✓ API is working correctly!")
        else:
            print("ERROR: Still no content/parts")
    else:
        print("ERROR: No candidates in response")
        
except Exception as e:
    print(f"ERROR: {e}")
    import traceback
    traceback.print_exc()


Testing Gemini API with CORRECTED max_output_tokens...
Response received successfully!

Finish reason: FinishReason.STOP

Generated text (695 chars):
Technology has become an inseparable extension of modern human existence, a powerful force that reshapes how we live, work, and connect. At its best, it acts as a great equalizer, democratizing access to information and bridging geographical divides with instantaneous communication. Yet, this same connectivity introduces complex challenges, from navigating issues of privacy and digital well-being to grappling with the rapid pace of change that can leave some behind. Ultimately, technology is a double-edged sword, a tool whose ultimate impact—whether it builds a more inclusive future or deepens existing divisions—is continually shaped by the choices we make in how we create and wield it.

✓ API is working correctly!


In [42]:
# Cell 4: Load Semantic Similarity Model
print("Loading sentence transformer for semantic similarity...")

semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

print(f"✓ Semantic model loaded")
print(f"  Model: all-MiniLM-L6-v2")

Loading sentence transformer for semantic similarity...


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1254.30it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✓ Semantic model loaded
  Model: all-MiniLM-L6-v2


In [ ]:
# Cell 5: Helper Functions - Classifier Prediction
def get_classifier_predictions(text):
    """
    Get classifier predictions for a text.
    Returns: (probs, predicted_class, confidence)
    
    IMPORTANT: Internal model uses 0-indexed classes (0, 1, 2)
    But project uses 1-indexed labels (Class 1, 2, 3):
      - Class 1 (Human)       → probs[0], pred_class=0
      - Class 2 (AI-written)  → probs[1], pred_class=1  
      - Class 3 (AI-mimicry)  → probs[2], pred_class=2
    
    When displaying, add +1: "Class {pred_class+1}"
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.softmax(logits, dim=1)[0].cpu().numpy()
    
    predicted_class = np.argmax(probs)
    confidence = probs[predicted_class]
    
    return probs, predicted_class, confidence

def get_human_probability(text):
    """Get P(Human) = P(Class 1) for a text."""
    probs, _, _ = get_classifier_predictions(text)
    return probs[0]

def get_log_human_probability(text):
    """Get log(P(Human)) for better numerical stability when P(Human) is very small."""
    probs, _, _ = get_classifier_predictions(text)
    return np.log(probs[0] + 1e-10)  # Add epsilon to avoid log(0)

# Test on a simple example
test_text = "Technology has become an integral part of modern life."
probs, pred_class, conf = get_classifier_predictions(test_text)
print(f"Test prediction:")
print(f"  P(Human):      {probs[0]:.8f}  [Class 1]")
print(f"  P(AI-written): {probs[1]:.8f}  [Class 2]")
print(f"  P(AI-mimicry): {probs[2]:.8f}  [Class 3]")
print(f"  log(P(Human)): {np.log(probs[0] + 1e-10):.8f}")
print(f"  Predicted: Class {pred_class+1} (internal idx={pred_class}, conf={conf:.4f})")


Test prediction:
  Human: 0.00253329
  AI-written: 0.99505049
  AI-mimicry: 0.00241629
  log(P(Human)): -5.97823811
  Predicted: Class 2 (conf=0.9951)


In [44]:
# Cell 6: [DEBUG] Helper Functions - Semantic Similarity
def get_semantic_similarity(text1, text2):
    """
    Compute cosine similarity between two texts using sentence embeddings.
    Returns: similarity score in [0, 1]
    """
    embeddings = semantic_model.encode([text1, text2])
    similarity = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
    return similarity

# Test
text1 = "The cat sat on the mat."
text2 = "A feline rested upon the rug."
text3 = "The weather is nice today."

sim_similar = get_semantic_similarity(text1, text2)
sim_different = get_semantic_similarity(text1, text3)

print(f"Semantic similarity test:")
print(f"  Similar texts: {sim_similar:.4f}")
print(f"  Different texts: {sim_different:.4f}")

Semantic similarity test:
  Similar texts: 0.5508
  Different texts: 0.0004


In [45]:
# Cell 7: Helper Functions - Perplexity Estimation
def estimate_perplexity(text):
    """
    Estimate perplexity using the classifier model as a proxy.
    Lower values = more fluent text.
    Note: This is a rough approximation since we don't have a dedicated language model.
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        # Use negative log likelihood as proxy
        # Higher confidence = lower perplexity
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)
        max_prob = torch.max(probs).item()
        
        # Rough perplexity estimate: inverse of confidence
        # This is not true perplexity but serves as fluency indicator
        perplexity = 1.0 / (max_prob + 1e-10)
    
    return perplexity

# Test
fluent_text = "The quick brown fox jumps over the lazy dog."
less_fluent = "The quick brown fox the lazy dog over jumps."

ppl_fluent = estimate_perplexity(fluent_text)
ppl_less = estimate_perplexity(less_fluent)

print(f"Perplexity test:")
print(f"  Fluent text: {ppl_fluent:.4f}")
print(f"  Less fluent: {ppl_less:.4f}")

Perplexity test:
  Fluent text: 1.0804
  Less fluent: 1.5800


In [57]:
# Cell 8: Fitness Function with Lagrangian Relaxation
def calculate_fitness(text, original_text, lambda_semantic=2.0, lambda_perplexity=0.5, 
                     semantic_threshold=0.85, perplexity_threshold=2.0):
    """
    Calculate fitness score with Lagrangian relaxation.
    
    Objective: Maximize P(Human) = Minimize P(AI)
    Constraints (soft penalties):
    - Semantic drift beyond threshold
    - Excessive perplexity
    
    Args:
        text: Candidate text
        original_text: Original text for semantic comparison
        lambda_semantic: Penalty weight for semantic drift
        lambda_perplexity: Penalty weight for perplexity
        semantic_threshold: Minimum acceptable similarity (0.85-0.90)
        perplexity_threshold: Maximum acceptable perplexity
    
    Returns:
        fitness: Higher is better
        components: Dict with breakdown
    """
    # Primary objective: maximize P(Human)
    # Use log probabilities to avoid saturation at 0.0000
    p_human = get_human_probability(text)
    log_p_human = get_log_human_probability(text)
    
    # Constraint 1: Semantic similarity
    similarity = get_semantic_similarity(text, original_text)
    semantic_penalty = 0.0
    if similarity < semantic_threshold:
        semantic_penalty = lambda_semantic * (semantic_threshold - similarity)
    
    # Constraint 2: Perplexity
    perplexity = estimate_perplexity(text)
    perplexity_penalty = 0.0
    if perplexity > perplexity_threshold:
        perplexity_penalty = lambda_perplexity * (perplexity - perplexity_threshold)
    
    # Final fitness (higher is better)
    # Use log space to preserve gradient when P(Human) is tiny
    fitness = log_p_human - semantic_penalty - perplexity_penalty
    
    components = {
        'fitness': fitness,
        'p_human': p_human,
        'log_p_human': log_p_human,
        'similarity': similarity,
        'perplexity': perplexity,
        'semantic_penalty': semantic_penalty,
        'perplexity_penalty': perplexity_penalty
    }
    
    return fitness, components

# Test
original = "Technology has become an integral part of modern life."
candidate = "Tech has become a key part of everyday life."

fitness, comp = calculate_fitness(candidate, original)
print(f"Fitness test (HIGH PRECISION):")
print(f"  Fitness: {fitness:.8f}")
print(f"  P(Human): {comp['p_human']:.8f}")
print(f"  log(P(Human)): {comp['log_p_human']:.8f}")
print(f"  Similarity: {comp['similarity']:.4f}")
print(f"  Perplexity: {comp['perplexity']:.4f}")
print(f"  Semantic penalty: {comp['semantic_penalty']:.8f}")
print(f"  Perplexity penalty: {comp['perplexity_penalty']:.8f}")


Fitness test (HIGH PRECISION):
  Fitness: -4.76383018
  P(Human): 0.00941421
  log(P(Human)): -4.66553497
  Similarity: 0.8009
  Perplexity: 1.0135
  Semantic penalty: 0.09829509
  Perplexity penalty: 0.00000000


In [ ]:
# Cell 9: SHAP-based Token Saliency
def get_token_saliency(text, target_class=1):
    """
    Compute token-level saliency using gradient-based approach.
    
    Args:
        text: Input text
        target_class: Which class to compute saliency for (1=Human, 2=AI-written, 3=AI-mimicry)
    
    Returns:
        tokens: List of tokens
        saliency_scores: Importance score for each token
        top_k_indices: Indices of most important tokens
    """
    model.train()  # Enable gradients
    
    # Tokenize
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    # Get embeddings and enable gradients
    embeddings = model.get_base_model().distilbert.embeddings.word_embeddings(input_ids)
    embeddings.requires_grad = True
    
    # Forward pass with custom embeddings
    outputs = model.get_base_model().distilbert(
        inputs_embeds=embeddings,
        attention_mask=attention_mask
    )
    
    # Get logits
    pooled_output = outputs[0][:, 0]  # CLS token
    logits = model.get_base_model().classifier(pooled_output)
    
    # Focus on target class
    target_logit = logits[0, target_class - 1]  # Adjust for 0-indexing
    
    # Backward pass
    target_logit.backward()
    
    # Get gradients
    saliency = embeddings.grad.abs().sum(dim=-1).squeeze().cpu().numpy()
    
    # Convert to tokens
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    
    # Get top-k indices (excluding special tokens)
    valid_indices = [i for i, tok in enumerate(tokens) 
                    if tok not in ['[CLS]', '[SEP]', '[PAD]']]
    
    # Sort by saliency
    token_saliency = [(i, tokens[i], saliency[i]) for i in valid_indices]
    token_saliency.sort(key=lambda x: x[2], reverse=True)
    
    # Get top 20% indices
    top_k = max(3, len(valid_indices) // 5)
    top_k_indices = [t[0] for t in token_saliency[:top_k]]
    
    model.eval()  # Disable gradients
    
    return tokens, saliency, top_k_indices, token_saliency

# Test
test_text = "Technology has become an integral part of modern life, shaping how we work."
tokens, saliency, top_indices, token_sal = get_token_saliency(test_text)

print(f"Token saliency test:")
print(f"  Total tokens: {len(tokens)}")
print(f"  Top {len(top_indices)} mutable tokens:")
for idx, token, sal in token_sal[:len(top_indices)]:
    print(f"    [{idx}] {token}: {sal:.4f}")

Token saliency test:
  Total tokens: 17
  Top 3 mutable tokens:
    [1] technology: 24.1031
    [8] modern: 22.2101
    [5] integral: 17.7286


In [48]:
# Cell 10: Gemini-based Text Mutation
def mutate_with_gemini(text, mutation_type="rhythm", temperature=0.9):
    """
    Use Gemini to mutate text while preserving meaning.
    
    Args:
        text: Original text
        mutation_type: Type of mutation (rhythm, vocabulary, structure, irregularity)
        temperature: Sampling temperature
    
    Returns:
        mutated_text: Modified text
    """
    mutation_prompts = {
        "rhythm": "slightly change the sentence rhythm and flow",
        "vocabulary": "swap some words with natural synonyms",
        "structure": "adjust sentence structure while keeping meaning",
        "irregularity": "introduce subtle stylistic irregularities that sound human"
    }
    
    prompt = f"""Rewrite this paragraph to {mutation_prompts.get(mutation_type, mutation_prompts['rhythm'])} while preserving the core meaning.

Original paragraph:
{text}

Rules:
- Keep the main message and information intact
- Make the writing feel more natural and less polished
- Do NOT add spelling errors or break grammar
- Avoid words like: tapestry, delve, testament, integral, unprecedented
- Output ONLY the rewritten paragraph, nothing else

Rewritten paragraph:""".strip()
    
    try:
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=prompt,
            config={
                "temperature": temperature,
                "top_p": 0.95,
                "max_output_tokens": 8192,
            }
        )
        
        if response.candidates and response.candidates[0].content.parts:
            mutated = response.candidates[0].content.parts[0].text.strip()
            # Remove quotes if present
            if mutated.startswith('"') and mutated.endswith('"'):
                mutated = mutated[1:-1]
            return mutated
        else:
            return text  # Return original if generation fails
            
    except Exception as e:
        print(f"  Mutation error: {e}")
        return text

# Test
test_text = "Technology has become an integral part of modern life."
mutated = mutate_with_gemini(test_text, "rhythm")
print(f"Mutation test:")
print(f"  Original: {test_text}")
print(f"  Mutated:  {mutated}")


Mutation test:
  Original: Technology has become an integral part of modern life.
  Mutated:  Technology is a huge part of how we live these days.


In [ ]:
# Cell 11: Generate Initial Population
def generate_initial_population(original_text, population_size=10):
    """
    Generate initial population from original text.
    - First individual: original text (copied to population/1.txt)
    - Rest: Gemini-generated variations with high temperature
    
    Args:
        original_text: The source text from input.txt
        population_size: Total population size (default 10)
    
    Returns:
        population: List of text candidates
    """
    print(f"Generating initial population of {population_size}...")
    
    # Clear population directory
    if POPULATION_DIR.exists():
        shutil.rmtree(POPULATION_DIR)
    POPULATION_DIR.mkdir(exist_ok=True)
    
    population = []
    
    # First individual: original text
    population.append(original_text)
    with open(POPULATION_DIR / "1.txt", "w", encoding="utf-8") as f:
        f.write(original_text)
    print(f"  [1/{population_size}] Original text saved")
    
    # Generate variations with high diversity
    prompt_template = """Generate a paragraph that is SIMILAR in meaning to this one, but written differently:

{text}

Requirements:
- Keep the same core message and topic
- Write it in a slightly different style or voice
- Change sentence structure and word choices
- Make it sound natural and fluent
- {words_min}-{words_max} words
- Output ONLY the paragraph, nothing else

New paragraph:"""

    word_count = len(original_text.split())
    words_min = max(50, int(word_count * 0.8))
    words_max = int(word_count * 1.2)
    
    for i in range(2, population_size + 1):
        prompt = prompt_template.format(
            text=original_text,
            words_min=words_min,
            words_max=words_max
        )
        
        try:
            response = client.models.generate_content(
                model=MODEL_NAME,
                contents=prompt,
                config={
                    "temperature": 1.0,  # High diversity
                    "top_p": 0.95,
                    "max_output_tokens": 8192,
                }
            )
            
            if response.candidates and response.candidates[0].content.parts:
                new_text = response.candidates[0].content.parts[0].text.strip()
                # Remove quotes if present
                if new_text.startswith('"') and new_text.endswith('"'):
                    new_text = new_text[1:-1]
                
                population.append(new_text)
                
                # Save to file
                with open(POPULATION_DIR / f"{i}.txt", "w", encoding="utf-8") as f:
                    f.write(new_text)
                
                print(f"  [{i}/{population_size}] Generated variation")
                
                # Rate limiting
                time.sleep(1)
            else:
                # Debug: Show why generation failed
                print(f"  [{i}/{population_size}] Generation failed!")
                if response.candidates:
                    print(f"      Finish reason: {response.candidates[0].finish_reason}")
                    if hasattr(response.candidates[0], 'safety_ratings'):
                        print(f"      Safety ratings: {response.candidates[0].safety_ratings}")
                else:
                    print(f"      No candidates in response")
                print(f"      Using original text instead")
                population.append(original_text)
                
        except Exception as e:
            print(f"  [{i}/{population_size}] Error: {e}, using original")
            population.append(original_text)
            time.sleep(2)
    
    print(f"Initial population generated: {len(population)} individuals")
    return population


In [50]:
# Cell 12: Local Search with Simulated Annealing
def local_search(text, original_text, max_iterations=5, initial_temp=0.1):
    """
    Perform local search on a candidate text.
    Uses simulated annealing to escape local optima.
    
    Args:
        text: Current candidate text
        original_text: Original text for fitness calculation
        max_iterations: Number of local search steps
        initial_temp: Initial temperature for simulated annealing
    
    Returns:
        best_text: Improved text
        best_fitness: Fitness of best text
    """
    current_text = text
    current_fitness, _ = calculate_fitness(current_text, original_text)
    best_text = current_text
    best_fitness = current_fitness
    
    mutation_types = ["rhythm", "vocabulary", "structure", "irregularity"]
    
    for iteration in range(max_iterations):
        # Temperature decay
        temp = initial_temp * (0.8 ** iteration)
        
        # Try mutation
        mutation_type = np.random.choice(mutation_types)
        candidate = mutate_with_gemini(current_text, mutation_type, temperature=0.7)
        candidate_fitness, _ = calculate_fitness(candidate, original_text)
        
        # Accept if better, or with probability based on temperature
        delta = candidate_fitness - current_fitness
        if delta > 0 or np.random.random() < np.exp(delta / temp):
            current_text = candidate
            current_fitness = candidate_fitness
            
            # Update best
            if current_fitness > best_fitness:
                best_text = current_text
                best_fitness = current_fitness
        
        # Rate limiting
        time.sleep(0.5)
    
    return best_text, best_fitness


In [51]:
# Cell 13: Evolution Operators
def selection(population, fitnesses, top_k=3):
    """Select top-k individuals."""
    sorted_indices = np.argsort(fitnesses)[::-1]
    selected_indices = sorted_indices[:top_k]
    selected = [population[i] for i in selected_indices]
    selected_fitnesses = [fitnesses[i] for i in selected_indices]
    return selected, selected_fitnesses, selected_indices

def crossover(parent1, parent2):
    """
    Crossover two parents by blending their styles.
    Uses Gemini to create a hybrid.
    """
    prompt = f"""Create a new paragraph that blends the writing style of these two paragraphs while keeping similar meaning:

Paragraph 1:
{parent1}

Paragraph 2:
{parent2}

Requirements:
- Combine stylistic elements from both paragraphs
- Maintain coherent meaning
- Create something that feels like a natural blend
- Output ONLY the blended paragraph

Blended paragraph:""".strip()
    
    try:
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=prompt,
            config={
                "temperature": 0.8,
                "top_p": 0.95,
                "max_output_tokens": 8192,
            }
        )
        
        if response.candidates and response.candidates[0].content.parts:
            child = response.candidates[0].content.parts[0].text.strip()
            if child.startswith('"') and child.endswith('"'):
                child = child[1:-1]
            return child
        else:
            return parent1
            
    except Exception as e:
        print(f"  Crossover error: {e}")
        return parent1

def mutate(text, mutation_rate=0.3):
    """
    Mutate text with given probability.
    """
    if np.random.random() < mutation_rate:
        mutation_type = np.random.choice(["rhythm", "vocabulary", "structure", "irregularity"])
        return mutate_with_gemini(text, mutation_type)
    return text

print("Evolution operators defined")


Evolution operators defined


In [52]:
# Cell 14: Log Generation to File
def log_generation(generation_num, population, fitnesses, original_text, filepath=None):
    """
    Log generation details to file.
    
    Args:
        generation_num: Generation number
        population: List of candidate texts
        fitnesses: List of fitness scores
        original_text: Original text for reference
        filepath: Optional custom filepath
    """
    if filepath is None:
        filepath = EVOLUTION_DIR / f"evolution_{generation_num}.txt"
    
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(f"GENERATION {generation_num}\n")
        
        # Sort by fitness
        sorted_indices = np.argsort(fitnesses)[::-1]
        
        for rank, idx in enumerate(sorted_indices, 1):
            text = population[idx]
            fitness = fitnesses[idx]
            
            # Get detailed metrics
            _, components = calculate_fitness(text, original_text)
            probs, pred_class, conf = get_classifier_predictions(text)
            
            f.write(f"Rank {rank} | Individual {idx + 1}\n\n")
            f.write(f"Fitness: {fitness:.4f}\n")
            f.write(f"  - P(Human): {components['p_human']:.4f}\n")
            f.write(f"  - P(AI): {probs[1]:.4f}\n")
            f.write(f"  - P(AI-mimicry): {probs[2]:.4f}\n")
            f.write(f"  - Semantic similarity: {components['similarity']:.4f}\n")
            f.write(f"  - Perplexity: {components['perplexity']:.4f}\n")
            f.write(f"  - Predicted class: {pred_class+1} (conf={conf:.4f})\n")
            f.write(f"\nText:\n{text}\n")
            f.write(f"\n{'=' * 80}\n\n")
        
        # Summary statistics
        f.write(f"\nSUMMARY STATISTICS\n")
        f.write(f"Best fitness: {max(fitnesses):.4f}\n")
        f.write(f"Avg fitness:  {np.mean(fitnesses):.4f}\n")
        f.write(f"Worst fitness: {min(fitnesses):.4f}\n")
        f.write(f"Std dev:      {np.std(fitnesses):.4f}\n")
    
    print(f"Logged to {filepath.name}")

print("Logging function defined")

Logging function defined


In [58]:
# Cell 15: Main MATE Algorithm
def run_mate(original_text, num_generations, population_size=10, top_k_parents=3):
    """
    Run the Memetic Algorithm for Text Evolution (MATE).
    
    Args:
        original_text: Source text to evolve
        num_generations: Number of generations to run
        population_size: Size of population
        top_k_parents: Number of parents to select
    
    Returns:
        best_individual: Best text found
        best_fitness: Fitness of best text
        history: Evolution history
    """
    print("=== MEMETIC ALGORITHM FOR TEXT EVOLUTION (MATE) ===")
    print("  FITNESS IN LOG SPACE - Expect negative values!")
    print("  ELITISM DISABLED - Will re-enable after signal stabilizes\n")
    
    # Setup evolution directory
    if EVOLUTION_DIR.exists():
        shutil.rmtree(EVOLUTION_DIR)
    EVOLUTION_DIR.mkdir(exist_ok=True)
    
    # Initialize population
    print("\n[STEP 1] Initializing population...")
    population = generate_initial_population(original_text, population_size)
    
    # Evaluate initial population
    print("\n[STEP 2] Evaluating initial population...")
    fitnesses = []
    for i, text in enumerate(population, 1):
        fitness, comp = calculate_fitness(text, original_text)
        fitnesses.append(fitness)
        probs, _, _ = get_classifier_predictions(text)
        # HIGH PRECISION logging to see tiny differences
        print(f"  [{i}/{population_size}] Fitness={fitness:.8f}, P(Human)={probs[0]:.8f}")
        print(f"      ├─ log(P(Human))={comp['log_p_human']:.8f}")
        print(f"      ├─ Semantic penalty={comp['semantic_penalty']:.8f}")
        print(f"      └─ Perplexity penalty={comp['perplexity_penalty']:.8f}")
    
    # Log generation 0
    log_generation(0, population, fitnesses, original_text)
    
    # Track best ever
    best_fitness = max(fitnesses)
    best_individual = population[np.argmax(fitnesses)]
    history = []
    
    print(f"\n✓ Initial best fitness: {best_fitness:.8f}")
    
    # Evolution loop
    for gen in range(1, num_generations + 1):
        print(f"\nGENERATION {gen}/{num_generations}")
        
        # Selection
        print("\n[STEP 3] Selection...")
        parents, parent_fitnesses, parent_indices = selection(population, fitnesses, top_k_parents)
        print(f"  Selected top {len(parents)} parents")
        for i, (p_idx, fit) in enumerate(zip(parent_indices, parent_fitnesses), 1):
            probs, _, _ = get_classifier_predictions(parents[i-1])
            _, comp = calculate_fitness(parents[i-1], original_text)
            print(f"    Parent {i} (idx={p_idx+1}): Fitness={fit:.8f}, P(Human)={probs[0]:.8f}")
            print(f"        └─ Components: log(P)={comp['log_p_human']:.6f}, sem_pen={comp['semantic_penalty']:.6f}, ppl_pen={comp['perplexity_penalty']:.6f}")
        
        # Create offspring via crossover and mutation
        print("\n[STEP 4] Crossover & Mutation...")
        offspring = []
        
        # ELITISM TEMPORARILY DISABLED for debugging
        # When fitness signal is saturated, elitism clones dead individuals
        # Re-enable after fitness starts separating
        # offspring.append(parents[0])
        # print(f"  Elitism: Keeping best parent")
        print(f"  Elitism: DISABLED (debug mode - fitness signal needs to stabilize first)")
        
        # Generate ALL offspring through crossover + mutation (no elitism)
        while len(offspring) < population_size:
            # Select two random parents
            p1, p2 = np.random.choice(len(parents), size=2, replace=False)
            
            # Crossover
            child = crossover(parents[p1], parents[p2])
            time.sleep(1)
            
            # Mutation
            child = mutate(child, mutation_rate=0.3)
            
            offspring.append(child)
            print(f"  Generated offspring {len(offspring)}/{population_size}")
            time.sleep(0.5)
        
        # Local search on top 3 offspring
        print("\n[STEP 5] Local search on top offspring...")
        offspring_fitnesses = [calculate_fitness(text, original_text)[0] for text in offspring]
        top_offspring_indices = np.argsort(offspring_fitnesses)[::-1][:3]
        
        for i, idx in enumerate(top_offspring_indices, 1):
            print(f"  Local search on offspring {idx+1}...")
            improved_text, improved_fitness = local_search(
                offspring[idx], 
                original_text, 
                max_iterations=3
            )
            offspring[idx] = improved_text
            offspring_fitnesses[idx] = improved_fitness
            print(f"    Fitness: {offspring_fitnesses[idx]:.8f}")
        
        # Replace population
        population = offspring
        fitnesses = offspring_fitnesses
        
        # Update best
        gen_best_fitness = max(fitnesses)
        gen_best_individual = population[np.argmax(fitnesses)]
        
        if gen_best_fitness > best_fitness:
            best_fitness = gen_best_fitness
            best_individual = gen_best_individual
            print(f"\nNEW BEST FITNESS: {best_fitness:.8f}")
        
        # Log generation
        log_generation(gen, population, fitnesses, original_text)
        
        # Statistics
        history.append({
            'generation': gen,
            'best_fitness': gen_best_fitness,
            'avg_fitness': np.mean(fitnesses),
            'worst_fitness': min(fitnesses),
            'best_p_human': get_human_probability(gen_best_individual)
        })
        
        print(f"\n  Generation {gen} summary (HIGH PRECISION):")
        print(f"    Best fitness:  {gen_best_fitness:.8f}")
        print(f"    Avg fitness:   {np.mean(fitnesses):.8f}")
        print(f"    Worst fitness: {min(fitnesses):.8f}")
        print(f"    Best P(Human): {get_human_probability(gen_best_individual):.8f}")
    
    # Final summary
    print("\n" + "="*60)
    print("EVOLUTION COMPLETE")
    print("="*60)
    print(f"\nBest fitness achieved: {best_fitness:.8f}")
    probs, pred_class, conf = get_classifier_predictions(best_individual)
    print(f"Best individual:")
    print(f"  P(Human): {probs[0]:.8f}")
    print(f"  P(AI): {probs[1]:.8f}")
    print(f"  P(AI-mimicry): {probs[2]:.8f}")
    print(f"  Predicted: Class {pred_class+1} (conf={conf:.4f})")
    print(f"\nText:\n{best_individual}")
    
    return best_individual, best_fitness, history


In [59]:
# Cell 16: RUN MATE - Configuration
# Read input text
print("Reading input text...")
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    original_text = f.read().strip()

if not original_text:
    print("WARNING: input.txt is empty!")
    print("Please add your AI-generated paragraph to input.txt and run this cell again.")
else:
    print(f" Input text loaded ({len(original_text.split())} words)")
    print(f"\nOriginal text:")
    print(f"{original_text}")
    
    # Get initial prediction
    probs, pred_class, conf = get_classifier_predictions(original_text)
    print(f"\nInitial classification:")
    print(f"  P(Human): {probs[0]:.4f}")
    print(f"  P(AI): {probs[1]:.4f}")
    print(f"  P(AI-mimicry): {probs[2]:.4f}")
    print(f"  Predicted: Class {pred_class+1} (conf={conf:.4f})")
    
    # Set number of generations
    num_generations = int(input("\nEnter number of generations to run: "))
    print(f"\nWill run {num_generations} generations")

Reading input text...
 Input text loaded (154 words)

Original text:
The quiet hum of this neighborhood always felt like safety, a soft blanket of routine where the biggest scandal was usually a zoning dispute or an unkempt lawn. I honestly believed I knew the man living in the yellow bungalow across the street. He was the sort of person who returned borrowed tools early and bought Girl Scout cookies by the case, practically invisible in his decency. That illusion shattered the afternoon I accidentally opened his mail, a simple mix-up by the postman. The letter wasn't a utility bill or a birthday card; it was a heavily redacted government dossier with a photo of him looking twenty years younger and terrifyingly cold. Standing there on my porch, holding the envelope that felt suddenly heavy, I realized his gentle demeanor wasn't a personality trait. It was a camouflage, perfect and impenetrable, hiding a history that had no business existing between the library and the bakery.

Initial 

# Some Important Terms

1. 

In [60]:
# Cell 17: RUN MATE - Execute Evolution
# Run MATE algorithm
best_individual, best_fitness, history = run_mate(
    original_text=original_text,
    num_generations=num_generations,
    population_size=10,
    top_k_parents=3
)

# Save best individual
best_file = Path("best_individual.txt")
with open(best_file, "w", encoding="utf-8") as f:
    f.write(f"Best Individual from MATE Evolution\n")
    f.write(f"Fitness: {best_fitness:.4f}\n\n")
    
    probs, pred_class, conf = get_classifier_predictions(best_individual)
    f.write(f"Classification:\n")
    f.write(f"  P(Human): {probs[0]:.4f}\n")
    f.write(f"  P(AI): {probs[1]:.4f}\n")
    f.write(f"  P(AI-mimicry): {probs[2]:.4f}\n")
    f.write(f"  Predicted: Class {pred_class} (conf={conf:.4f})\n\n")
    f.write(f"Text:\n{best_individual}\n")

print(f"\nBest individual saved to {best_file}")

=== MEMETIC ALGORITHM FOR TEXT EVOLUTION (MATE) ===
  FITNESS IN LOG SPACE - Expect negative values!
  ELITISM DISABLED - Will re-enable after signal stabilizes


[STEP 1] Initializing population...
Generating initial population of 10...
  [1/10] Original text saved
  [2/10] Generated variation
  [3/10] Generated variation
  [4/10] Generated variation
  [5/10] Generated variation
  [6/10] Generated variation
  [7/10] Generated variation
  [8/10] Generated variation
  [9/10] Generated variation
  [10/10] Generated variation
✓ Initial population generated: 10 individuals

[STEP 2] Evaluating initial population...
  [1/10] Fitness=-14.04408646, P(Human)=0.00000080
      ├─ log(P(Human))=-14.04408646
      ├─ Semantic penalty=0.00000000
      └─ Perplexity penalty=0.00000000
  [2/10] Fitness=-14.26171398, P(Human)=0.00000070
      ├─ log(P(Human))=-14.17290115
      ├─ Semantic penalty=0.08881271
      └─ Perplexity penalty=0.00000000
  [3/10] Fitness=-13.46456242, P(Human)=0.00000172
    

After a few rounds of running cell 17, I kept getting [this output](results/1-always-0.md). This is quite problematic. It makes things meaningless if children are evolving from a parent who itself is 0% human! As such, I realise that while the algorithm may be okay, there are a few things I need to change, because rn the memetic algorithm has nothing to optimize!!! This makes local search / global search / whatever you want to call it, meaningless...




In [ ]:
# Cell 18: Visualize Evolution Progress
import matplotlib.pyplot as plt

# Plot fitness evolution
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Plot 1: Best and average fitness over generations
generations = [h['generation'] for h in history]
best_fitnesses = [h['best_fitness'] for h in history]
avg_fitnesses = [h['avg_fitness'] for h in history]

axes[0].plot(generations, best_fitnesses, 'b-o', label='Best Fitness', linewidth=2)
axes[0].plot(generations, avg_fitnesses, 'g--s', label='Avg Fitness', linewidth=2)
axes[0].set_xlabel('Generation', fontsize=12)
axes[0].set_ylabel('Fitness', fontsize=12)
axes[0].set_title('Fitness Evolution Over Generations', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Plot 2: P(Human) probability over generations
p_humans = [h['best_p_human'] for h in history]
axes[1].plot(generations, p_humans, 'r-o', label='P(Human) of Best', linewidth=2)
axes[1].axhline(y=0.5, color='k', linestyle='--', alpha=0.5, label='Decision Boundary')
axes[1].set_xlabel('Generation', fontsize=12)
axes[1].set_ylabel('P(Human)', fontsize=12)
axes[1].set_title('Human Probability of Best Individual', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([0, 1])

plt.tight_layout()
plt.savefig('mate_evolution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Visualization saved to mate_evolution.png")

In [ ]:
# Cell 19: Compare Original vs Best
print("COMPARISON: ORIGINAL vs BEST EVOLVED")

print("\n[ORIGINAL TEXT]")
print(original_text)
print()

probs_orig, pred_orig, conf_orig = get_classifier_predictions(original_text)
print(f"Classification:")
print(f"  P(Human): {probs_orig[0]:.4f}")
print(f"  P(AI): {probs_orig[1]:.4f}")
print(f"  P(AI-mimicry): {probs_orig[2]:.4f}")
print(f"  Predicted: Class {pred_orig+1} (conf={conf_orig:.4f})")

print("\n[BEST EVOLVED TEXT]")
print(best_individual)
print()

probs_best, pred_best, conf_best = get_classifier_predictions(best_individual)
print(f"Classification:")
print(f"  P(Human): {probs_best[0]:.4f}")
print(f"  P(AI): {probs_best[1]:.4f}")
print(f"  P(AI-mimicry): {probs_best[2]:.4f}")
print(f"  Predicted: Class {pred_best+1} (conf={conf_best:.4f})")

print("\n[IMPROVEMENTS]")
print(f"P(Human) change: {probs_orig[0]:.4f} → {probs_best[0]:.4f} (+{probs_best[0] - probs_orig[0]:.4f})")
print(f"P(AI) change: {probs_orig[1]:.4f} → {probs_best[1]:.4f} ({probs_best[1] - probs_orig[1]:.4f})")
print(f"Semantic similarity: {get_semantic_similarity(original_text, best_individual):.4f}")

if pred_best == 0:
    print("\nSUCCESS: Best individual is classified as HUMAN (Class 1)!")
else:
    print(f"\nStill classified as Class {pred_best+1}, but P(Human) improved by {(probs_best[0] - probs_orig[0]) * 100:.2f}%")